<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Box Plots**


Estimated time needed: **45** minutes


In this lab, you will focus on the visualization of data. The dataset will be provided through an RDBMS, and you will need to use SQL queries to extract the required data.


## Objectives


In this lab you will perform the following:


-   Visualize the distribution of data.

-   Visualize the relationship between two features.

-   Visualize data composition and comparisons using box plots.


### Setup: Connecting to the Database


#### 1. Download the Database File


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/QR9YeprUYhOoLafzlLspAw/survey-results-public.sqlite

--2025-06-13 15:55:32--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/QR9YeprUYhOoLafzlLspAw/survey-results-public.sqlite
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
200 OKequest sent, awaiting response... 
Length: 211415040 (202M) [application/octet-stream]
Saving to: ‘survey-results-public.sqlite.1’

          survey-re   0%[                    ]       0  --.-KB/s               

#### 2. Connect to the Database


**Install the needed libraries**


In [ ]:
!pip install pandas

In [ ]:
!pip install matplotlib

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Connect to the SQLite database
conn = sqlite3.connect('survey-results-public.sqlite')


## Demo: Basic SQL Queries


#### Demo 1: Count the Number of Rows in the Table


In [ ]:
QUERY = "SELECT COUNT(*) FROM main"
df = pd.read_sql_query(QUERY, conn)
print(df)


#### Demo 2: List All Tables


In [ ]:
QUERY = """
SELECT name as Table_Name 
FROM sqlite_master 
WHERE type = 'table'
"""
pd.read_sql_query(QUERY, conn)


#### Demo 3: Group Data by Age


In [ ]:
QUERY = """
SELECT Age, COUNT(*) as count 
FROM main 
GROUP BY Age 
ORDER BY Age
"""
df_age = pd.read_sql_query(QUERY, conn)
print(df_age)


## Visualizing Data


### Task 1: Visualizing the Distribution of Data


**1. Box Plot of `CompTotal` (Total Compensation)**


Use a box plot to analyze the distribution and outliers in total compensation.


In [ ]:
# your code goes here
QUERY = """
SELECT
    CASE
        WHEN YearsCodePro = 'Less than 1 year' THEN 0.5
        WHEN YearsCodePro = 'More than 50 years' THEN 51
        ELSE CAST(YearsCodePro AS REAL)
    END AS YearsCodePro,
    CompTotal
FROM main
WHERE YearsCodePro IS NOT NULL 
AND CompTotal IS NOT NULL 
AND CompTotal >= 0
AND CompTotal <= 500000
"""

df = pd.read_sql_query(QUERY, conn)

bins = [0, 1, 5, 10, 20, 100]
labels = ['<1 year', '1-5 years', '5-10 years', '10-20 years', '20+ years']
df['ExperienceGroup'] = pd.cut(df['YearsCodePro'], bins=bins, labels=labels, right=False)

plt.figure(figsize=(10, 6))
df.boxplot(column='CompTotal', by='ExperienceGroup', grid=True, patch_artist=True,
           boxprops=dict(facecolor='lightgreen', color='darkgreen'),
           medianprops=dict(color='red'))

plt.title('Total Compensation Distribution by Years of Professional Coding Experience')
plt.suptitle('')  # to remove the automatic 'Boxplot grouped by...' title
plt.xlabel('Years of Professional Coding Experience')
plt.ylabel('Total Compensation')
plt.xticks(rotation=45)
plt.show()


**2. Box Plot of Age (converted to numeric values)**


Convert the `Age` column into numerical values and visualize the distribution.


In [ ]:
# your code goes here
# SQL query
QUERY = """
SELECT 
  CASE
    WHEN Age = 'Under 18 years old' THEN 17
    WHEN Age = '18-24 years old' THEN 21
    WHEN Age = '25-34 years old' THEN 29
    WHEN Age = '35-44 years old' THEN 39
    WHEN Age = '45-54 years old' THEN 49
    WHEN Age = '55-64 years old' THEN 59
    WHEN Age = '65 or older' THEN 65
    ELSE NULL
  END AS AgeNum
FROM main
WHERE Age IS NOT NULL
"""

# Load data
df = pd.read_sql_query(QUERY, conn)

# Plot histogram of AgeNum
plt.figure(figsize=(8, 6))
plt.hist(df['AgeNum'], bins=range(15, 71, 10), color='skyblue', edgecolor='black', align='left')
plt.title('Distribution of Respondents by Age')
plt.xlabel('Age (Approximate Midpoints)')
plt.ylabel('Number of Respondents')
plt.xticks([17, 21, 29, 39, 49, 59, 65], rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

### Task 2: Visualizing Relationships in Data


**1. Box Plot of `CompTotal` Grouped by Age Groups:**


Visualize the distribution of compensation across different age groups.


In [ ]:
# your code goes here
QUERY = """SELECT 
  CASE
    WHEN Age = 'Under 18 years old' THEN 17
    WHEN Age = '18-24 years old' THEN 21
    WHEN Age = '25-34 years old' THEN 29
    WHEN Age = '35-44 years old' THEN 39
    WHEN Age = '45-54 years old' THEN 49
    WHEN Age = '55-64 years old' THEN 59
    WHEN Age = '65 or older' THEN 65
    ELSE NULL
  END AS AgeNum,
  CompTotal
FROM main
WHERE Age IS NOT NULL AND CompTotal IS NOT NULL AND CompTotal >= 0 AND CompTotal <= 500000"""

# Run SQL query
df = pd.read_sql_query(QUERY, conn)

# Plot box plot of compensation by AgeNum
plt.figure(figsize=(10, 6))
df.boxplot(column='CompTotal', by='AgeNum', grid=True, patch_artist=True,
           boxprops=dict(facecolor='lightblue', color='navy'),
           medianprops=dict(color='red'))

plt.title('Total Compensation by Age Group')
plt.suptitle('')  # Remove default title
plt.xlabel('Age (Midpoint of Age Range)')
plt.ylabel('Total Compensation')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


**2. Box Plot of `CompTotal` Grouped by Job Satisfaction (`JobSatPoints_6`):**


Examine how compensation varies based on job satisfaction levels.


In [ ]:
# your code goes here
# Run query
QUERY = """
SELECT
  JobSat,
  CAST(CompTotal AS REAL) AS CompTotal
FROM main
WHERE JobSat IS NOT NULL
  AND CompTotal IS NOT NULL
  AND CAST(CompTotal AS REAL) >= 0
  AND CAST(CompTotal AS REAL) <= 500000
"""

df = pd.read_sql_query(QUERY, conn)

# Sort JobSat categories for consistent order (optional)
df['JobSat'] = pd.Categorical(df['JobSat'], ordered=True)

plt.figure(figsize=(10, 6))
df.boxplot(column='CompTotal', by='JobSat', grid=True, patch_artist=True,
           boxprops=dict(facecolor='lightblue'),
           medianprops=dict(color='red'))

plt.title('Compensation by Job Satisfaction Level')
plt.suptitle('')
plt.xlabel('Job Satisfaction')
plt.ylabel('Total Compensation')
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.show()


### Task 3: Visualizing the Composition of Data


**1. Box Plot of `ConvertedCompYearly` for the Top 5 Developer Types:**


Analyze compensation across the top 5 developer roles.


In [ ]:
# your code goes here
QUERY = """SELECT
  DevType,
  CAST(CompTotal AS REAL) AS CompTotal
FROM main
WHERE DevType IS NOT NULL
  AND CompTotal IS NOT NULL
  AND CAST(CompTotal AS REAL) >= 0
  AND CAST(CompTotal AS REAL) <= 500000
"""

# Read from SQLite
df = pd.read_sql_query(QUERY, conn)

# Split DevType values (semicolon-separated list of roles)
df = df.assign(DevType=df['DevType'].str.split(';'))

# Explode into one row per role
df = df.explode('DevType')
df['DevType'] = df['DevType'].str.strip()

# Get top 5 most common roles
top_roles = df['DevType'].value_counts().head(5).index.tolist()

# Filter to those top 5 roles
df_top = df[df['DevType'].isin(top_roles)]

# Plot box plot of compensation by role
plt.figure(figsize=(10, 6))
df_top.boxplot(column='CompTotal', by='DevType', patch_artist=True,
               boxprops=dict(facecolor='lightgreen'),
               medianprops=dict(color='red'))

plt.title('Compensation by Top 5 Developer Roles')
plt.suptitle('')
plt.xlabel('Developer Role')
plt.ylabel('Total Compensation')
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.grid(axis='y')
plt.show()

**2. Box Plot of `CompTotal` for the Top 5 Countries:**


Analyze compensation across respondents from the top 5 countries.


In [ ]:
# your code goes here
QUERY = """SELECT
  Country,
  CAST(CompTotal AS REAL) AS CompTotal
FROM main
WHERE Country IS NOT NULL
  AND CompTotal IS NOT NULL
  AND CAST(CompTotal AS REAL) >= 0
  AND CAST(CompTotal AS REAL) <= 500000
"""
# Load data
df = pd.read_sql_query(QUERY, conn)

# Identify top 5 countries by count
top_countries = df['Country'].value_counts().head(5).index.tolist()

# Filter to those countries
df_top = df[df['Country'].isin(top_countries)]

# Create box plot
plt.figure(figsize=(10, 6))
df_top.boxplot(column='CompTotal', by='Country', patch_artist=True,
               boxprops=dict(facecolor='lightblue'),
               medianprops=dict(color='darkred'))

plt.title('Compensation by Country (Top 5 Respondent Countries)')
plt.suptitle('')
plt.xlabel('Country')
plt.ylabel('Total Compensation')
plt.xticks(rotation=30, ha='right')
plt.grid(axis='y')
plt.tight_layout()
plt.show()

### Task 4: Visualizing Comparison of Data


**1. Box Plot of CompTotal Across Employment Types:**


Analyze compensation for different employment types.


In [ ]:
# your code goes here
QUERY = """
SELECT
  Employment,
  CAST(CompTotal AS REAL) AS CompTotal
FROM main
WHERE Employment IS NOT NULL
  AND CompTotal IS NOT NULL
  AND CAST(CompTotal AS REAL) >= 0
  AND CAST(CompTotal AS REAL) <= 500000
"""
# Load data
df = pd.read_sql_query(QUERY, conn)

# Split multiple employment types (delimited by ';')
df['Employment'] = df['Employment'].str.split(';')

# Explode so each employment type is its own row
df = df.explode('Employment')

# Strip leading/trailing whitespace
df['Employment'] = df['Employment'].str.strip()

# Order employment types by frequency
employment_order = df['Employment'].value_counts().index.tolist()
df['Employment'] = pd.Categorical(df['Employment'], categories=employment_order, ordered=True)

# Plot
plt.figure(figsize=(12, 6))
df.boxplot(column='CompTotal', by='Employment', patch_artist=True,
           boxprops=dict(facecolor='lightgray'),
           medianprops=dict(color='red'))

plt.title('Compensation by Employment Type')
plt.suptitle('')
plt.xlabel('Employment Type')
plt.ylabel('Total Compensation (USD)')
plt.xticks(rotation=30, ha='right')
plt.grid(axis='y')
plt.tight_layout()
plt.show()


**2. Box Plot of `YearsCodePro` by Job Satisfaction (`JobSatPoints_6`):**


Examine the distribution of professional coding years by job satisfaction levels.


In [ ]:
# your code goes here
QUERY = """SELECT
  CASE
    WHEN YearsCodePro = 'Less than 1 year' THEN 0.5
    WHEN YearsCodePro = 'More than 50 years' THEN 51
    ELSE CAST(YearsCodePro AS REAL)
  END AS YearsCodePro,
  JobSat
FROM main
WHERE YearsCodePro IS NOT NULL
  AND JobSat IS NOT NULL
"""

# Load cleaned data
df = pd.read_sql_query(QUERY, conn)

# Just to be sure, reset your JobSat categories with correct labels
job_sat_order = [
    'Very dissatisfied',
    'Slightly dissatisfied',
    'Neither satisfied nor dissatisfied',
    'Slightly satisfied',
    'Very satisfied'
]

# Your current JobSat is numeric (5, 8, 10, 6, 9 etc.), so let's check unique JobSat values:
print(df['JobSat'].unique())

# If JobSat is numeric codes, either map them to labels or treat as categorical directly
# For example, if JobSat is on a 1-10 scale, plot it directly (or bin it)

# Let's try a simple boxplot grouping by JobSat as numeric categories:
plt.figure(figsize=(10, 6))
df.boxplot(column='YearsCodePro', by='JobSat', patch_artist=True,
           boxprops=dict(facecolor='lightsteelblue'),
           medianprops=dict(color='red'))

plt.title('Professional Coding Experience by Job Satisfaction')
plt.suptitle('')
plt.xlabel('Job Satisfaction (numeric)')
plt.ylabel('Years of Professional Coding Experience')
plt.xticks(rotation=30, ha='right')
plt.grid(axis='y')
plt.tight_layout()
plt.show()


### Final Step: Close the Database Connection


After completing the lab, close the connection to the SQLite database:


In [ ]:
conn.close()

## Summary


In this lab, you used box plots to visualize various aspects of the dataset, focusing on:

- Visualize distributions of compensation and age.

- Explore relationships between compensation, job satisfaction, and professional coding experience.

- Analyze data composition across developer roles and countries.

- Compare compensation across employment types and satisfaction levels.

Box plots provided clear insights into the spread, outliers, and central tendencies of various features in the dataset.


## Authors:
Ayushi Jain


### Other Contributors:
- Rav Ahuja
- Lakshmi Holla
- Malika


<!--## Change Log
|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|               
|2024-10-07|1.2|Madhusudan Moole|Reviewed and updated lab|                                                                                      
|2024-10-06|1.0|Raghul Ramesh|Created lab|-->


Copyright © IBM Corporation. All rights reserved.
